In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi

Sun Oct  8 20:36:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq huggingface-hub==0.16.4 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [4]:
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/cuda/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
notebook_login()

In [8]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [19]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)

In [10]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

In [14]:
model = model.eval()

In [15]:
def format_prompt(prompt: str, system_prompt: str) -> str:
    return f"""
{system_prompt}
{prompt}
""".strip()

In [16]:
def generate_response(prompt: str, max_new_tokens: int = 128) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.0001,
            generation_config=generation_config,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)

In [20]:
%%time

# text = """
# What is twitter
# """

prompt = f"""
Write detailed notes based on this transcript of a lecture. 
Use markdown. 
Use several layers of nested headings and bold and italics but don't use any headings past heading 3. 
You should also include a detailed table of content.
Don't include any sponsored ads

6
00:00:10,980 --> 00:00:15,660
 today we're going to talk

7
00:00:13,080 --> 00:00:17,899
about Transformer neural networks and

8
00:00:15,660 --> 00:00:21,359
they're going to be clearly explained

12
00:00:25,679 --> 00:00:32,159
right now people are going bonkers about

13
00:00:28,739 --> 00:00:35,100
something called chat GPT for example

14
00:00:32,159 --> 00:00:38,039
our friend statsquatch might type

15
00:00:35,100 --> 00:00:42,558
something into chat GPT like

16
00:00:38,039 --> 00:00:44,479
right and awesome song in the style of

17
00:00:42,558 --> 00:00:47,820
statquest

18
00:00:44,479 --> 00:00:49,738
translation it's done with a transform

19
00:00:47,820 --> 00:00:52,320
ER

20
00:00:49,738 --> 00:00:55,919
anyway there's a lot to be said about

21
00:00:52,320 --> 00:00:57,539
how chat GPT works but fundamentally it

22
00:00:55,920 --> 00:00:58,980
is based on something called a

23
00:00:57,539 --> 00:01:01,739
Transformer

24
00:00:58,979 --> 00:01:04,259
so in this stat Quest we're going to

25
00:01:01,738 --> 00:01:06,118
show you how a Transformer works one

26
00:01:04,260 --> 00:01:08,880
step at a time

27
00:01:06,118 --> 00:01:10,978
specifically we're going to focus on how

28
00:01:08,879 --> 00:01:13,039
a Transformer neural network can

29
00:01:10,978 --> 00:01:17,700
translate a simple English sentence

30
00:01:13,040 --> 00:01:20,640
let's go into Spanish vamos

31
00:01:17,700 --> 00:01:22,618
now since a Transformer is a type of

32
00:01:20,640 --> 00:01:24,840
neural network and neural networks

33
00:01:22,618 --> 00:01:25,859
usually only have numbers for input

34
00:01:24,840 --> 00:01:28,439
values

35
00:01:25,859 --> 00:01:30,500
the first thing we need to do is find a

36
00:01:28,438 --> 00:01:32,819
way to turn the input and output words

37
00:01:30,500 --> 00:01:35,219
into numbers

38
00:01:32,819 --> 00:01:38,158
there are a lot of ways to convert words

39
00:01:35,219 --> 00:01:40,200
into numbers but for neural networks one

40
00:01:38,159 --> 00:01:43,200
of the most commonly used methods is

41
00:01:40,200 --> 00:01:45,180
called word embedding the main idea of

42
00:01:43,200 --> 00:01:48,478
word embedding is to use a relatively

43
00:01:45,180 --> 00:01:50,700
simple neural network that has one input

44
00:01:48,478 --> 00:01:54,000
for every word and symbol in the

45
00:01:50,700 --> 00:01:56,700
vocabulary that you want to use in this

46
00:01:54,000 --> 00:01:59,099
case we have a super simple vocabulary

47
00:01:56,700 --> 00:02:02,100
that allows us to input short phrases

48
00:01:59,099 --> 00:02:05,158
like let's go and to go

49
00:02:02,099 --> 00:02:08,159
and we have an input for this symbol EOS

50
00:02:05,159 --> 00:02:11,219
which stands for end of sentence or end

51
00:02:08,159 --> 00:02:13,859
of sequence because the vocabulary can

52
00:02:11,219 --> 00:02:17,340
be a mix of words word fragments and

53
00:02:13,860 --> 00:02:19,379
symbols we call each input a token

54
00:02:17,340 --> 00:02:21,239
the inputs are then connected to

55
00:02:19,378 --> 00:02:23,639
something called an activation function

56
00:02:21,239 --> 00:02:25,439
and in this example we have two

57
00:02:23,639 --> 00:02:28,019
activation functions

58
00:02:25,439 --> 00:02:30,840
and each connection multiplies the input

59
00:02:28,020 --> 00:02:33,659
value by something called a weight

60
00:02:30,840 --> 00:02:34,560
hey Josh where do these numbers come

61
00:02:33,659 --> 00:02:37,020
from

62
00:02:34,560 --> 00:02:38,759
great question Squatch and we'll answer

63
00:02:37,020 --> 00:02:41,280
it in just a bit

64
00:02:38,759 --> 00:02:43,859
for now let's just see how we convert

65
00:02:41,280 --> 00:02:46,500
the word let's into numbers

66
00:02:43,860 --> 00:02:49,860
first we put a 1 into the input for

67
00:02:46,500 --> 00:02:51,479
let's and then put zeros into all of the

68
00:02:49,860 --> 00:02:53,760
other inputs

69
00:02:51,479 --> 00:02:55,560
now we multiply the inputs by their

70
00:02:53,759 --> 00:02:57,359
weights on the connections to the

71
00:02:55,560 --> 00:03:00,420
activation functions

72
00:02:57,360 --> 00:03:05,280
for example the input for let's is one

73
00:03:00,419 --> 00:03:07,859
so we multiply 1.87 by 1 to get 1.87

74
00:03:05,280 --> 00:03:12,860
going to the activation function on the

75
00:03:07,860 --> 00:03:16,319
left and we multiply 0.09 by 1 to get

76
00:03:12,860 --> 00:03:17,459
0.09 going to the activation function on

77
00:03:16,318 --> 00:03:20,219
the right

78
00:03:17,459 --> 00:03:22,680
in contrast if the input value for the

79
00:03:20,219 --> 00:03:27,060
word 2 is 0

80
00:03:22,680 --> 00:03:29,280
then we multiply negative 1.45 by 0 to

81
00:03:27,060 --> 00:03:30,719
get 0 going to the activation function

82
00:03:29,280 --> 00:03:34,919
on the left

83
00:03:30,719 --> 00:03:37,979
and we multiply 1.50 by 0 to get 0 going

84
00:03:34,919 --> 00:03:40,738
to the activation function on the right

85
00:03:37,979 --> 00:03:43,199
in other words when an input value is 0

86
00:03:40,739 --> 00:03:46,920
then it only sends zeros to the

87
00:03:43,199 --> 00:03:50,219
activation functions and that means to

88
00:03:46,919 --> 00:03:52,919
go and the EOS symbol all just send

89
00:03:50,219 --> 00:03:55,620
zeros to the activation functions

90
00:03:52,919 --> 00:03:57,598
and only the weight values for let's end

91
00:03:55,620 --> 00:04:00,299
up at the activation functions because

92
00:03:57,598 --> 00:04:04,018
its input value is 1.

93
00:04:00,299 --> 00:04:06,540
so in this case 1.87 goes to the

94
00:04:04,019 --> 00:04:10,140
activation function on the left

95
00:04:06,539 --> 00:04:12,000
and 0.09 goes to the activation function

96
00:04:10,139 --> 00:04:14,818
on the right

97
00:04:12,000 --> 00:04:17,279
in this example the activation functions

98
00:04:14,818 --> 00:04:19,620
themselves are just identity functions

99
00:04:17,279 --> 00:04:21,599
meaning the output values are the same

100
00:04:19,620 --> 00:04:24,478
as the input values

101
00:04:21,600 --> 00:04:26,639
in other words if the input value or

102
00:04:24,478 --> 00:04:30,779
x-axis coordinate for the activation

103
00:04:26,639 --> 00:04:33,720
function on the left is 1.87 then the

104
00:04:30,779 --> 00:04:36,059
output value the y-axis coordinate will

105
00:04:33,720 --> 00:04:38,400
also be 1.87

106
00:04:36,060 --> 00:04:42,360
likewise because the input to the

107
00:04:38,399 --> 00:04:46,019
activation function on the right is 0.09

108
00:04:42,360 --> 00:04:50,879
the output is also 0.09

109
00:04:46,019 --> 00:04:53,159
thus these output values 1.87 and 0.09

110
00:04:50,879 --> 00:04:55,639
are the numbers that represent the word

111
00:04:53,160 --> 00:04:58,259
leads bam

112
00:04:55,639 --> 00:05:00,300
likewise if we want to convert the word

113
00:04:58,259 --> 00:05:04,560
go into numbers

114
00:05:00,300 --> 00:05:07,379
we set the input value for go to 1. and

115
00:05:04,560 --> 00:05:11,000
all of the other inputs to zero

116
00:05:07,379 --> 00:05:13,978
and we end up with negative 0.78 and

117
00:05:11,000 --> 00:05:15,660
0.27 as the numbers that represent the

118
00:05:13,978 --> 00:05:18,180
word go

119
00:05:15,660 --> 00:05:21,300
and that is how we use word embedding to

120
00:05:18,180 --> 00:05:24,300
convert our input phrase let's go into

121
00:05:21,300 --> 00:05:26,939
numbers bam

122
00:05:24,300 --> 00:05:28,860
note there is a lot more to say about

123
00:05:26,939 --> 00:05:30,600
word embedding so if you're interested

124
00:05:28,860 --> 00:05:33,538
check out the quest

125
00:05:30,600 --> 00:05:35,340
also note before we move on I want to

126
00:05:33,538 --> 00:05:38,038
point out two things

127
00:05:35,339 --> 00:05:41,159
first we reuse the same word embedding

128
00:05:38,038 --> 00:05:43,319
Network for each input word or symbol

129
00:05:41,160 --> 00:05:45,000
in other words the weights in the

130
00:05:43,319 --> 00:05:47,639
network for let's

131
00:05:45,000 --> 00:05:49,500
are the exact same as the weights in the

132
00:05:47,639 --> 00:05:51,960
network for go

133
00:05:49,500 --> 00:05:54,720
this means that regardless of how long

134
00:05:51,959 --> 00:05:56,579
the input sentence is we just copy and

135
00:05:54,720 --> 00:05:59,280
use the exact same word embedding

136
00:05:56,579 --> 00:06:01,740
Network for each word or symbol

137
00:05:59,279 --> 00:06:04,739
and this gives us flexibility to handle

138
00:06:01,740 --> 00:06:06,840
input sentences with different lengths

139
00:06:04,740 --> 00:06:08,939
the second thing I want to mention is

140
00:06:06,839 --> 00:06:10,319
that all of these weights and all of the

141
00:06:08,939 --> 00:06:12,959
other weights we're going to talk about

142
00:06:10,319 --> 00:06:15,418
in this Quest are determined using

143
00:06:12,959 --> 00:06:17,698
something called back propagation

144
00:06:15,418 --> 00:06:20,939
to get a sense of what back propagation

145
00:06:17,699 --> 00:06:23,400
does let's imagine we had this data and

146
00:06:20,939 --> 00:06:25,800
we wanted to fit a line to it

147
00:06:23,399 --> 00:06:28,258
back propagation would start with a line

148
00:06:25,800 --> 00:06:31,079
that has a random value for the y-axis

149
00:06:28,259 --> 00:06:31,979
intercept and a random value for the

150
00:06:31,079 --> 00:06:35,339
slope

151
00:06:31,978 --> 00:06:37,680
and then using an iterative process back

152
00:06:35,339 --> 00:06:40,560
propagation would change the y-axis

153
00:06:37,680 --> 00:06:43,939
intercept and slope one step at a time

154
00:06:40,560 --> 00:06:46,500
until it found the optimal values

155
00:06:43,939 --> 00:06:49,079
likewise in the context of neural

156
00:06:46,500 --> 00:06:50,399
networks each weight starts out as a

157
00:06:49,079 --> 00:06:53,038
random number

158
00:06:50,399 --> 00:06:55,079
but when we train the Transformer with

159
00:06:53,038 --> 00:06:56,818
English phrases and known Spanish

160
00:06:55,079 --> 00:06:59,639
translations

161
00:06:56,819 --> 00:07:02,520
back propagation optimizes these values

162
00:06:59,639 --> 00:07:04,079
one step at a time and results in these

163
00:07:02,519 --> 00:07:07,318
final weights

164
00:07:04,079 --> 00:07:09,478
also just to be clear the process of

165
00:07:07,319 --> 00:07:11,819
optimizing the weights is also called

166
00:07:09,478 --> 00:07:14,339
training bam

167
00:07:11,819 --> 00:07:16,860
note there is a lot more to be said

168
00:07:14,339 --> 00:07:18,418
about training and back propagation so

169
00:07:16,860 --> 00:07:19,560
if you're interested check out the

170
00:07:18,418 --> 00:07:22,019
quests

171
00:07:19,560 --> 00:07:24,538
now because the word embedding networks

172
00:07:22,019 --> 00:07:26,279
are taking up the whole screen let's

173
00:07:24,538 --> 00:07:29,399
shrink them down and put them in the

174
00:07:26,279 --> 00:07:32,279
corner okay and now that we know how to

175
00:07:29,399 --> 00:07:34,198
convert words into numbers let's talk

176
00:07:32,279 --> 00:07:37,978
about word order

177
00:07:34,199 --> 00:07:43,080
for example if Norm said Squatch eats

178
00:07:37,978 --> 00:07:47,038
pizza then squash might say yum

179
00:07:43,079 --> 00:07:52,019
in contrast if Norm said Pizza eats

180
00:07:47,038 --> 00:07:54,180
squash then squash might say yikes

181
00:07:52,019 --> 00:07:56,399
so these two phrases

182
00:07:54,180 --> 00:08:00,060
Squatch eats Pizza

183
00:07:56,399 --> 00:08:03,299
and pizza eats squash use the exact same

184
00:08:00,060 --> 00:08:06,300
words but have very different meanings

185
00:08:03,300 --> 00:08:09,060
so keeping track of word order is super

186
00:08:06,300 --> 00:08:11,098
important so let's talk about positional

187
00:08:09,060 --> 00:08:13,379
encoding which is a technique that

188
00:08:11,098 --> 00:08:14,459
Transformers use to keep track of word

189
00:08:13,379 --> 00:08:16,500
order

190
00:08:14,459 --> 00:08:18,839
we'll start by showing how to add

191
00:08:16,500 --> 00:08:21,478
positional encoding to the first phrase

192
00:08:18,839 --> 00:08:23,939
Squatch eats Pizza

193
00:08:21,478 --> 00:08:26,098
note there are a bunch of ways to do

194
00:08:23,939 --> 00:08:28,680
positional encoding but we're just going

195
00:08:26,098 --> 00:08:31,019
to talk about one popular method

196
00:08:28,680 --> 00:08:34,019
that said the first thing we do is

197
00:08:31,019 --> 00:08:36,478
convert the words squash eats pizza into

198
00:08:34,019 --> 00:08:38,519
numbers using word embedding

199
00:08:36,479 --> 00:08:40,979
in this example we've got a new

200
00:08:38,519 --> 00:08:43,379
vocabulary and we're creating four word

201
00:08:40,979 --> 00:08:46,379
embedding values per word

202
00:08:43,379 --> 00:08:48,838
however in practice people often create

203
00:08:46,379 --> 00:08:50,820
hundreds or even thousands of embedding

204
00:08:48,839 --> 00:08:52,920
values per word

205
00:08:50,820 --> 00:08:54,959
now we add a set of numbers that

206
00:08:52,919 --> 00:08:58,019
correspond to word order to the

207
00:08:54,958 --> 00:09:00,479
embedding values for each word hey Josh

208
00:08:58,019 --> 00:09:02,459
where do the numbers that correspond to

209
00:09:00,480 --> 00:09:04,920
word order come from

210
00:09:02,458 --> 00:09:07,079
in this case the numbers that represent

211
00:09:04,919 --> 00:09:10,439
the word order come from a sequence of

212
00:09:07,080 --> 00:09:12,839
alternating sine and cosine squiggles

213
00:09:10,440 --> 00:09:15,660
each squiggle gives a specific position

214
00:09:12,839 --> 00:09:18,720
values for each word's embeddings

215
00:09:15,659 --> 00:09:20,939
for example the y-axis values on the

216
00:09:18,720 --> 00:09:23,278
green squiggle give us position encoding

217
00:09:20,940 --> 00:09:24,380
values for the first embeddings for each

218
00:09:23,278 --> 00:09:27,299
word

219
00:09:24,379 --> 00:09:29,939
specifically for the first word which

220
00:09:27,299 --> 00:09:32,579
has an x-axis coordinate all the way to

221
00:09:29,940 --> 00:09:34,500
the left of the green squiggle the

222
00:09:32,580 --> 00:09:37,980
position value for the first embedding

223
00:09:34,500 --> 00:09:39,839
is the y-axis coordinate zero

224
00:09:37,980 --> 00:09:42,120
the position value for the second

225
00:09:39,839 --> 00:09:44,760
embedding comes from the orange squiggle

226
00:09:42,120 --> 00:09:46,919
and the y-axis coordinate on the orange

227
00:09:44,759 --> 00:09:48,899
squiggle that corresponds to the first

228
00:09:46,919 --> 00:09:51,599
word is one

229
00:09:48,899 --> 00:09:53,720
likewise the blue squiggle which is more

230
00:09:51,600 --> 00:09:56,399
spread out than the first two squiggles

231
00:09:53,720 --> 00:09:58,379
gives us the position value for the

232
00:09:56,399 --> 00:10:00,539
third embedding value which for the

233
00:09:58,379 --> 00:10:03,120
first word is zero

234
00:10:00,539 --> 00:10:05,639
lastly the red squiggle gives us the

235
00:10:03,120 --> 00:10:09,120
position value for the fourth embedding

236
00:10:05,639 --> 00:10:11,519
which for the first word is one

237
00:10:09,120 --> 00:10:14,039
thus the position values for the first

238
00:10:11,519 --> 00:10:16,259
word come from the corresponding y-axis

239
00:10:14,039 --> 00:10:18,899
coordinates on the squiggles

240
00:10:16,259 --> 00:10:21,899
now to get the position values for the

241
00:10:18,899 --> 00:10:23,580
second word we simply use the y-axis

242
00:10:21,899 --> 00:10:25,980
coordinates on the squiggles that

243
00:10:23,580 --> 00:10:27,660
correspond to the x-axis coordinate for

244
00:10:25,980 --> 00:10:30,360
the second word

245
00:10:27,659 --> 00:10:33,179
lastly to get the position values for

246
00:10:30,360 --> 00:10:34,740
the third word we use the y-axis

247
00:10:33,179 --> 00:10:37,019
coordinates on the squiggles that

248
00:10:34,740 --> 00:10:38,700
correspond to the x-axis coordinate for

249
00:10:37,019 --> 00:10:41,278
the third word

250
00:10:38,700 --> 00:10:43,620
note because the sine and cosine

251
00:10:41,278 --> 00:10:45,480
squiggles are repetitive it's possible

252
00:10:43,620 --> 00:10:48,778
that two words might get the same

253
00:10:45,480 --> 00:10:51,180
position or y-axis values

254
00:10:48,778 --> 00:10:54,120
for example the second and third words

255
00:10:51,179 --> 00:10:55,739
both got negative 0.9 for the first

256
00:10:54,120 --> 00:10:58,500
position value

257
00:10:55,740 --> 00:11:01,259
however because the squiggles get wider

258
00:10:58,500 --> 00:11:03,480
for larger embedding positions and the

259
00:11:01,259 --> 00:11:05,338
more embedding values we have then the

260
00:11:03,480 --> 00:11:08,159
wider the squiggles get

261
00:11:05,339 --> 00:11:10,440
then even with a repeat value here and

262
00:11:08,159 --> 00:11:13,338
there we end up with a unique sequence

263
00:11:10,440 --> 00:11:16,500
of position values for each word

264
00:11:13,339 --> 00:11:19,079
thus each input word ends up with a

265
00:11:16,500 --> 00:11:21,299
unique sequence of position values

266
00:11:19,078 --> 00:11:23,399
now all we have to do is add the

267
00:11:21,299 --> 00:11:25,979
position values to the embedding values

268
00:11:23,399 --> 00:11:28,200
and we end up with the word embeddings

269
00:11:25,980 --> 00:11:31,879
plus positional encoding for the whole

270
00:11:28,200 --> 00:11:33,660
sentence Squatch eats Pizza

271
00:11:31,879 --> 00:11:36,299
yum

272
00:11:33,659 --> 00:11:40,679
note if we reverse the order of the

273
00:11:36,299 --> 00:11:42,599
input words to be Pizza eats squash then

274
00:11:40,679 --> 00:11:45,359
the embeddings for the first and third

275
00:11:42,600 --> 00:11:47,700
words get swapped but the positional

276
00:11:45,360 --> 00:11:51,000
values for the first second and third

277
00:11:47,700 --> 00:11:53,519
word stay the same and when we add the

278
00:11:51,000 --> 00:11:55,919
positional values to the embeddings

279
00:11:53,519 --> 00:11:58,980
we end up with new positional encoding

280
00:11:55,919 --> 00:12:01,439
for the first and third words and the

281
00:11:58,980 --> 00:12:02,579
second word since it didn't move stays

282
00:12:01,440 --> 00:12:05,160
the same

283
00:12:02,578 --> 00:12:07,139
thus positional encoding allows a

284
00:12:05,159 --> 00:12:10,139
Transformer to keep track of word order

285
00:12:07,139 --> 00:12:10,139
bam

286
00:12:10,200 --> 00:12:14,220
now let's go back to our simple example

287
00:12:12,419 --> 00:12:16,919
where we are just trying to translate

288
00:12:14,220 --> 00:12:19,019
the English sentence let's go

289
00:12:16,919 --> 00:12:20,219
and add position values to the word

290
00:12:19,019 --> 00:12:22,320
embeddings

291
00:12:20,220 --> 00:12:25,860
the first embedding for the first word

292
00:12:22,320 --> 00:12:28,920
Let's gets zero and the second embedding

293
00:12:25,860 --> 00:12:33,120
gets one and the first embedding for the

294
00:12:28,919 --> 00:12:36,240
second word go gets a negative 0.9 and

295
00:12:33,120 --> 00:12:38,578
the second embedding gets 0.4

296
00:12:36,240 --> 00:12:42,778
and now we just do the math to get the

297
00:12:38,578 --> 00:12:45,179
positional encoding for both words bam

298
00:12:42,778 --> 00:12:47,458
now because we're going to need all the

299
00:12:45,179 --> 00:12:50,458
space we can get let's consolidate the

300
00:12:47,458 --> 00:12:52,859
math in the diagram and let the sine and

301
00:12:50,458 --> 00:12:54,778
cosine and plus symbols represent the

302
00:12:52,860 --> 00:12:56,940
positional encoding

303
00:12:54,778 --> 00:12:59,700
now that we know how to keep track of

304
00:12:56,940 --> 00:13:01,920
each word's position let's talk about

305
00:12:59,700 --> 00:13:04,620
how a Transformer keeps track of the

306
00:13:01,919 --> 00:13:07,379
relationships among words

307
00:13:04,620 --> 00:13:10,919
for example if the input sentence was

308
00:13:07,379 --> 00:13:15,000
this the pizza came out of the oven and

309
00:13:10,919 --> 00:13:18,659
it tasted good then this word it could

310
00:13:15,000 --> 00:13:22,139
refer to pizza or potentially it could

311
00:13:18,659 --> 00:13:24,719
refer to the word oven Josh I've heard

312
00:13:22,139 --> 00:13:28,200
of good tasting pizza but never a good

313
00:13:24,720 --> 00:13:29,940
tasting oven I know Squatch that's why

314
00:13:28,200 --> 00:13:32,579
it's important that the Transformer

315
00:13:29,940 --> 00:13:35,700
correctly Associates the word it with

316
00:13:32,578 --> 00:13:37,379
pizza the good news is that Transformers

317
00:13:35,700 --> 00:13:39,540
have something called self-attention

318
00:13:37,379 --> 00:13:41,820
which is a mechanism to correctly

319
00:13:39,539 --> 00:13:43,019
associate the word ID with the word

320
00:13:41,820 --> 00:13:46,260
Pizza

321
00:13:43,019 --> 00:13:48,778
in general terms self-attention works by

322
00:13:46,259 --> 00:13:51,000
seeing how similar each word is to all

323
00:13:48,778 --> 00:13:54,059
of the words in the sentence including

324
00:13:51,000 --> 00:13:56,278
itself for example self-attention

325
00:13:54,059 --> 00:13:59,159
calculates the similarity between the

326
00:13:56,278 --> 00:14:02,458
first word the and all of the words in

327
00:13:59,159 --> 00:14:04,620
the sentence including itself

328
00:14:02,458 --> 00:14:06,599
and self-attention calculates these

329
00:14:04,620 --> 00:14:07,980
similarities for every word in the

330
00:14:06,600 --> 00:14:10,139
sentence

331
00:14:07,980 --> 00:14:12,300
once the similarities are calculated

332
00:14:10,139 --> 00:14:15,120
they are used to determine how the

333
00:14:12,299 --> 00:14:17,519
Transformer encodes each word

334
00:14:15,120 --> 00:14:19,980
for example if you looked at a lot of

335
00:14:17,519 --> 00:14:22,379
sentences about pizza and the word ID

336
00:14:19,980 --> 00:14:24,060
was more commonly associated with pizza

337
00:14:22,379 --> 00:14:26,820
than oven

338
00:14:24,059 --> 00:14:28,919
then the similarity score for pizza will

339
00:14:26,820 --> 00:14:31,019
cause it to have a larger impact on how

340
00:14:28,919 --> 00:14:32,278
the word ID is encoded by the

341
00:14:31,019 --> 00:14:33,839
Transformer

342
00:14:32,278 --> 00:14:36,120
bam

343
00:14:33,839 --> 00:14:38,399
and now that we know the main ideas of

344
00:14:36,120 --> 00:14:39,659
how self-attention Works let's look at

345
00:14:38,399 --> 00:14:41,759
the details

346
00:14:39,659 --> 00:14:43,860
so let's go back to our simple example

347
00:14:41,759 --> 00:14:47,399
where we had just added positional

348
00:14:43,860 --> 00:14:49,680
encoding to the words let's and go

349
00:14:47,399 --> 00:14:51,659
the first thing we do is multiply the

350
00:14:49,679 --> 00:14:54,838
position encoded values for the word

351
00:14:51,659 --> 00:14:57,059
let's by a pair of weights and we add

352
00:14:54,839 --> 00:14:58,740
those products together to get Negative

353
00:14:57,059 --> 00:15:00,719
1.0

354
00:14:58,740 --> 00:15:04,680
then we do the same thing with a

355
00:15:00,720 --> 00:15:07,199
different pair of weights to get 3.7

356
00:15:04,679 --> 00:15:09,299
we do this twice because we started out

357
00:15:07,198 --> 00:15:12,479
with two position encoded values that

358
00:15:09,299 --> 00:15:15,240
represent the word leads and after doing

359
00:15:12,480 --> 00:15:17,940
the math two times we still have two

360
00:15:15,240 --> 00:15:20,339
values representing the word leads

361
00:15:17,940 --> 00:15:22,920
Josh I don't get it

362
00:15:20,339 --> 00:15:25,620
if we want two values to represent let's

363
00:15:22,919 --> 00:15:26,819
why don't we just use the two values we

364
00:15:25,620 --> 00:15:29,100
started with

365
00:15:26,820 --> 00:15:33,360
that's a great question Squatch and

366
00:15:29,100 --> 00:15:36,240
we'll answer it in a little bit grr

367
00:15:33,360 --> 00:15:38,519
anyway for now just know that we have

368
00:15:36,240 --> 00:15:40,500
these two new values to represent the

369
00:15:38,519 --> 00:15:43,198
word let's and in Transformer

370
00:15:40,500 --> 00:15:45,778
terminology we call them query values

371
00:15:43,198 --> 00:15:48,599
and now that we have query values for

372
00:15:45,778 --> 00:15:50,939
the word let's use them to calculate the

373
00:15:48,600 --> 00:15:52,079
similarity between itself and the word

374
00:15:50,940 --> 00:15:54,540
go

375
00:15:52,078 --> 00:15:57,239
and we do this by creating two new

376
00:15:54,539 --> 00:15:59,338
values just like we did for the query to

377
00:15:57,240 --> 00:16:01,500
represent the word let's

378
00:15:59,339 --> 00:16:03,720
and we create two new values to

379
00:16:01,500 --> 00:16:06,600
represent the word go

380
00:16:03,720 --> 00:16:07,860
both sets of new values are called key

381
00:16:06,600 --> 00:16:09,600
values

382
00:16:07,860 --> 00:16:13,019
and we use them to calculate

383
00:16:09,600 --> 00:16:15,180
similarities with the query for let's

384
00:16:13,019 --> 00:16:17,759
one way to calculate similarities

385
00:16:15,179 --> 00:16:20,399
between the query and the keys is to

386
00:16:17,759 --> 00:16:22,860
calculate something called a DOT product

387
00:16:20,399 --> 00:16:25,198
for example in order to calculate the

388
00:16:22,860 --> 00:16:27,240
dot product similarity between the query

389
00:16:25,198 --> 00:16:29,698
and key for let's

390
00:16:27,240 --> 00:16:30,959
we simply multiply each pair of numbers

391
00:16:29,698 --> 00:16:34,859
together

392
00:16:30,958 --> 00:16:37,198
and then add the products to get 11.7

393
00:16:34,860 --> 00:16:39,300
likewise we can calculate the dot

394
00:16:37,198 --> 00:16:41,819
product similarity between the query for

395
00:16:39,299 --> 00:16:43,799
let's and the key for go

396
00:16:41,820 --> 00:16:45,060
by multiplying the pairs of numbers

397
00:16:43,799 --> 00:16:47,399
together

398
00:16:45,059 --> 00:16:49,198
and adding the products to get negative

399
00:16:47,399 --> 00:16:51,659
2.6

400
00:16:49,198 --> 00:16:54,258
the relatively large similarity value

401
00:16:51,659 --> 00:16:57,838
for let's relative to itself

402
00:16:54,259 --> 00:17:00,659
11.7 compared to the relatively small

403
00:16:57,839 --> 00:17:03,060
value for lets relative to the word go

404
00:17:00,659 --> 00:17:05,699
negative 2.6

405
00:17:03,059 --> 00:17:09,058
tells us that let's is much more similar

406
00:17:05,699 --> 00:17:11,519
to itself than it is to the word go

407
00:17:09,058 --> 00:17:13,980
that said if you remember the example

408
00:17:11,519 --> 00:17:15,418
where the word it could relate to pizza

409
00:17:13,980 --> 00:17:17,880
or oven

410
00:17:15,419 --> 00:17:20,280
the word it should have a relatively

411
00:17:17,880 --> 00:17:22,980
large similarity value with respect to

412
00:17:20,279 --> 00:17:25,019
the word Pizza since it refers to pizza

413
00:17:22,980 --> 00:17:27,419
and not oven

414
00:17:25,019 --> 00:17:29,639
note there's a lot to be said about

415
00:17:27,419 --> 00:17:31,500
calculating similarities in this context

416
00:17:29,640 --> 00:17:34,140
and the dot product so if you're

417
00:17:31,500 --> 00:17:36,660
interested check out the quests anyway

418
00:17:34,140 --> 00:17:39,480
since let's is much more similar to

419
00:17:36,660 --> 00:17:41,580
itself than it is to the word go

420
00:17:39,480 --> 00:17:43,798
then we want let's to have more

421
00:17:41,579 --> 00:17:45,058
influence on its encoding than the word

422
00:17:43,798 --> 00:17:47,220
go

423
00:17:45,058 --> 00:17:48,960
and we do this by first running the

424
00:17:47,220 --> 00:17:51,660
similarities course through something

425
00:17:48,960 --> 00:17:54,779
called a soft Max function

426
00:17:51,660 --> 00:17:56,820
the main idea of a soft Max function is

427
00:17:54,779 --> 00:17:59,700
that it preserves the order of the input

428
00:17:56,819 --> 00:18:02,399
values from low to high and translates

429
00:17:59,700 --> 00:18:04,019
them into numbers between 0 and 1 that

430
00:18:02,400 --> 00:18:06,059
add up to one

431
00:18:04,019 --> 00:18:08,639
so we can think of the output of the

432
00:18:06,058 --> 00:18:11,099
softmax function as a way to determine

433
00:18:08,640 --> 00:18:13,860
what percentage of each input word we

434
00:18:11,099 --> 00:18:16,500
should use to encode the word let's

435
00:18:13,859 --> 00:18:18,959
in this case because let's is so much

436
00:18:16,500 --> 00:18:21,240
more similar to itself than the word go

437
00:18:18,960 --> 00:18:23,819
we'll use one hundred percent of the

438
00:18:21,240 --> 00:18:26,519
word let's to encode less

439
00:18:23,819 --> 00:18:28,259
and zero percent of the word go to

440
00:18:26,519 --> 00:18:30,599
encode the word let's

441
00:18:28,259 --> 00:18:32,640
note there's a lot more to be said about

442
00:18:30,599 --> 00:18:34,798
the soft Max function so if you're

443
00:18:32,640 --> 00:18:37,980
interested check out the quest

444
00:18:34,798 --> 00:18:40,500
anyway because we want 100 of the word

445
00:18:37,980 --> 00:18:43,140
let's to encode let's

446
00:18:40,500 --> 00:18:46,140
we create two more values that will

447
00:18:43,140 --> 00:18:47,400
cleverly call values to represent the

448
00:18:46,140 --> 00:18:49,440
word let's

449
00:18:47,400 --> 00:18:51,780
and scale the values that represent

450
00:18:49,440 --> 00:18:54,120
let's by 1.0

451
00:18:51,779 --> 00:18:55,798
then we create two values to represent

452
00:18:54,119 --> 00:18:59,339
the word go

453
00:18:55,798 --> 00:19:02,819
and scale those values by 0.0

454
00:18:59,339 --> 00:19:05,159
lastly we add the scaled values together

455
00:19:02,819 --> 00:19:08,099
and these sums which combine separate

456
00:19:05,160 --> 00:19:10,798
encodings for both input words let's and

457
00:19:08,099 --> 00:19:13,859
go relative to their similarity to Let's

458
00:19:10,798 --> 00:19:15,960
are the self-attention values for leads

459
00:19:13,859 --> 00:19:18,119
bam

460
00:19:15,960 --> 00:19:20,700
now that we have self-attention values

461
00:19:18,119 --> 00:19:22,979
for the word let's it's time to

462
00:19:20,700 --> 00:19:25,319
calculate them for the word go

463
00:19:22,980 --> 00:19:28,919
the good news is that we don't need to

464
00:19:25,319 --> 00:19:31,139
recalculate the keys and values instead

465
00:19:28,919 --> 00:19:33,660
all we need to do is create the query

466
00:19:31,140 --> 00:19:35,460
that represents the word go

467
00:19:33,660 --> 00:19:37,679
and do the math

468
00:19:35,460 --> 00:19:40,079
by first calculating the similarity

469
00:19:37,679 --> 00:19:42,900
scores between the new query and the

470
00:19:40,079 --> 00:19:44,879
keys and then run the similarity scores

471
00:19:42,900 --> 00:19:47,220
through a softmax

472
00:19:44,880 --> 00:19:49,380
and then scale the values

473
00:19:47,220 --> 00:19:51,538
and then add them together

474
00:19:49,380 --> 00:19:53,760
and we end up with the self-attention

475
00:19:51,538 --> 00:19:56,700
values for go

476
00:19:53,759 --> 00:19:59,519
note before we move on I want to point

477
00:19:56,700 --> 00:20:01,679
out a few details about self-attention

478
00:19:59,519 --> 00:20:03,960
first the weights that we use to

479
00:20:01,679 --> 00:20:06,780
calculate the self-attention queries are

480
00:20:03,960 --> 00:20:09,720
the exact same for let's and go

481
00:20:06,779 --> 00:20:11,038
in other words this example uses one set

482
00:20:09,720 --> 00:20:13,679
of weights for calculating

483
00:20:11,038 --> 00:20:16,019
self-attention queries regardless of how

484
00:20:13,679 --> 00:20:18,840
many words are in the input

485
00:20:16,019 --> 00:20:21,418
likewise we reuse the sets of weights

486
00:20:18,839 --> 00:20:23,879
for calculating self-attention keys and

487
00:20:21,419 --> 00:20:26,038
values for each input word

488
00:20:23,880 --> 00:20:28,679
this means that no matter how many words

489
00:20:26,038 --> 00:20:31,379
are input into the Transformer

490
00:20:28,679 --> 00:20:34,080
we just reuse the same sets of weights

491
00:20:31,380 --> 00:20:35,460
for self-attention queries keys and

492
00:20:34,079 --> 00:20:37,500
values

493
00:20:35,460 --> 00:20:39,660
the other thing I want to point out is

494
00:20:37,500 --> 00:20:42,179
that we can calculate the queries keys

495
00:20:39,660 --> 00:20:43,320
and values for each word at the same

496
00:20:42,179 --> 00:20:45,419
time

497
00:20:43,319 --> 00:20:47,819
in other words we don't have to

498
00:20:45,419 --> 00:20:50,520
calculate the query key and value for

499
00:20:47,819 --> 00:20:51,839
the first word first before moving on to

500
00:20:50,519 --> 00:20:53,819
the second word

501
00:20:51,839 --> 00:20:55,819
and because we can do all of the

502
00:20:53,819 --> 00:20:58,019
computation at the same time

503
00:20:55,819 --> 00:21:01,019
Transformers can take advantage of

504
00:20:58,019 --> 00:21:03,119
parallel Computing and run fast

505
00:21:01,019 --> 00:21:05,579
now that we understand the details of

506
00:21:03,119 --> 00:21:07,199
how self-attention Works let's shrink it

507
00:21:05,579 --> 00:21:11,399
down so we can keep building our

508
00:21:07,200 --> 00:21:13,919
Transformer bam Josh you forgot

509
00:21:11,400 --> 00:21:16,320
something if we want two values to

510
00:21:13,919 --> 00:21:18,419
represent let's why don't we just use

511
00:21:16,319 --> 00:21:19,740
the two position encoded values we

512
00:21:18,419 --> 00:21:22,440
started with

513
00:21:19,740 --> 00:21:25,019
first the new self-attention values for

514
00:21:22,440 --> 00:21:27,240
each word contain input from all of the

515
00:21:25,019 --> 00:21:30,298
other words and this helps give each

516
00:21:27,240 --> 00:21:32,819
word context and this can help establish

517
00:21:30,298 --> 00:21:33,960
how each word in the input is related to

518
00:21:32,819 --> 00:21:37,079
the others

519
00:21:33,960 --> 00:21:39,480
also if we think of this unit with its

520
00:21:37,079 --> 00:21:42,298
weights for calculating queries keys and

521
00:21:39,480 --> 00:21:45,179
values as a self-attention cell

522
00:21:42,298 --> 00:21:46,918
then in order to correctly establish how

523
00:21:45,179 --> 00:21:49,259
words are related in complicated

524
00:21:46,919 --> 00:21:51,840
sentences and paragraphs

525
00:21:49,259 --> 00:21:54,839
we can create a stack of self-attention

526
00:21:51,839 --> 00:21:57,000
cells each with its own sets of Weights

527
00:21:54,839 --> 00:21:59,220
that we apply to the position encoded

528
00:21:57,000 --> 00:22:02,279
values for each word to capture

529
00:21:59,220 --> 00:22:04,500
different relationships among the words

530
00:22:02,279 --> 00:22:06,538
in the manuscript that first describes

531
00:22:04,500 --> 00:22:08,460
Transformers they stacked eight

532
00:22:06,538 --> 00:22:10,798
self-attention cells and they called

533
00:22:08,460 --> 00:22:14,700
this multi-head attention

534
00:22:10,798 --> 00:22:15,798
why eight instead of 12 or 16 I have no

535
00:22:14,700 --> 00:22:17,580
idea

536
00:22:15,798 --> 00:22:20,279
bam

537
00:22:17,579 --> 00:22:23,099
okay going back to our simple example

538
00:22:20,279 --> 00:22:25,558
with only one self-attention cell

539
00:22:23,099 --> 00:22:27,178
there's one more thing we need to do to

540
00:22:25,558 --> 00:22:30,178
encode the input

541
00:22:27,179 --> 00:22:32,280
we take the position encoded values

542
00:22:30,179 --> 00:22:35,220
and add them to the self-attention

543
00:22:32,279 --> 00:22:37,440
values these bypasses are called

544
00:22:35,220 --> 00:22:40,860
residual connections and they make it

545
00:22:37,440 --> 00:22:43,380
easier to train complex neural networks

546
00:22:40,859 --> 00:22:45,479
by allowing the self-attention layer to

547
00:22:43,380 --> 00:22:47,760
establish relationships among the input

548
00:22:45,480 --> 00:22:49,620
words without having to also preserve

549
00:22:47,759 --> 00:22:51,240
the word embedding and positioning

550
00:22:49,619 --> 00:22:52,619
coding information

551
00:22:51,240 --> 00:22:55,019
bam

552
00:22:52,619 --> 00:22:58,019
and that's all we need to do to encode

553
00:22:55,019 --> 00:23:02,519
the input for this simple Transformer

554
00:22:58,019 --> 00:23:04,859
double bam note this simple Transformer

555
00:23:02,519 --> 00:23:07,700
only contains the parts required for

556
00:23:04,859 --> 00:23:11,819
encoding the input word embedding

557
00:23:07,700 --> 00:23:13,740
positional encoding self-attention and

558
00:23:11,819 --> 00:23:15,839
residual connections

559
00:23:13,740 --> 00:23:19,579
these four features allow the

560
00:23:15,839 --> 00:23:23,399
Transformer to encode words into numbers

561
00:23:19,579 --> 00:23:26,399
encode the positions of the words encode

562
00:23:23,400 --> 00:23:29,220
the relationships among the words and

563
00:23:26,400 --> 00:23:32,159
relatively easily and quickly train in

564
00:23:29,220 --> 00:23:34,558
parallel that said there are lots of

565
00:23:32,159 --> 00:23:36,600
extra things we can add to a Transformer

566
00:23:34,558 --> 00:23:40,980
and we'll talk about those at the end of

567
00:23:36,599 --> 00:23:43,859
this Quest bam so now that we've encoded

568
00:23:40,980 --> 00:23:46,620
the English input phrase let's go it's

569
00:23:43,859 --> 00:23:49,019
time to decode it into Spanish

570
00:23:46,619 --> 00:23:52,019
in other words the first part of a

571
00:23:49,019 --> 00:23:54,359
transformer is called an encoder and now

572
00:23:52,019 --> 00:23:55,619
it's time to create the second part A

573
00:23:54,359 --> 00:23:58,918
decoder

574
00:23:55,619 --> 00:24:02,158
the decoder just like the encoder starts

575
00:23:58,919 --> 00:24:04,140
with word embedding however this time we

576
00:24:02,159 --> 00:24:06,659
create embedding values for the output

577
00:24:04,140 --> 00:24:12,240
vocabulary which consists of the Spanish

578
00:24:06,659 --> 00:24:14,039
words ear vamos e and the EOS end of

579
00:24:12,240 --> 00:24:16,798
sequence token

580
00:24:14,038 --> 00:24:19,259
now because we just finished encoding

581
00:24:16,798 --> 00:24:21,658
the English sentence let's go

582
00:24:19,259 --> 00:24:25,140
the decoder starts with embedding values

583
00:24:21,659 --> 00:24:27,600
for the EOS token in this case we're

584
00:24:25,140 --> 00:24:30,120
using the EOS token to start the

585
00:24:27,599 --> 00:24:32,519
decoding because that is a common way to

586
00:24:30,119 --> 00:24:34,798
initialize the process of decoding the

587
00:24:32,519 --> 00:24:37,259
encoded input sentence

588
00:24:34,798 --> 00:24:40,200
however sometimes you'll see people use

589
00:24:37,259 --> 00:24:43,019
SOS for startup sentence or start of

590
00:24:40,200 --> 00:24:46,440
sequence to initialize the process

591
00:24:43,019 --> 00:24:48,658
Josh starting with SOS makes more sense

592
00:24:46,440 --> 00:24:51,179
to me then you can do it that way

593
00:24:48,659 --> 00:24:53,400
Squatch I'm just saying a lot of people

594
00:24:51,179 --> 00:24:57,480
start with EOS

595
00:24:53,400 --> 00:24:58,798
anyway we plug in 1 for Eos and zero for

596
00:24:57,480 --> 00:25:00,839
everything else

597
00:24:58,798 --> 00:25:04,579
and do the math

598
00:25:00,839 --> 00:25:07,798
and we end up with 2.70 and negative

599
00:25:04,579 --> 00:25:10,558
1.34 as the numbers that represent the

600
00:25:07,798 --> 00:25:12,599
EOS token bam

601
00:25:10,558 --> 00:25:14,460
now let's shrink the word embedding down

602
00:25:12,599 --> 00:25:16,199
to make more space

603
00:25:14,460 --> 00:25:17,519
so that we can add the positional

604
00:25:16,200 --> 00:25:20,460
encoding

605
00:25:17,519 --> 00:25:22,558
note these are the exact same sine and

606
00:25:20,460 --> 00:25:25,558
cosine squiggles that we used when we

607
00:25:22,558 --> 00:25:28,139
encoded the input and since the EOS

608
00:25:25,558 --> 00:25:30,298
token is in the first position with two

609
00:25:28,140 --> 00:25:31,860
embeddings we just add those two

610
00:25:30,298 --> 00:25:36,538
position values

611
00:25:31,859 --> 00:25:38,219
and we get 2.70 and negative 0.34 as the

612
00:25:36,538 --> 00:25:42,720
position and word embedding values

613
00:25:38,220 --> 00:25:45,298
representing the EOS token bam now let's

614
00:25:42,720 --> 00:25:47,278
consolidate the math in the diagram

615
00:25:45,298 --> 00:25:50,038
and before we move on to the next step

616
00:25:47,278 --> 00:25:51,720
let's review a key concept from when we

617
00:25:50,038 --> 00:25:54,480
encoded the input

618
00:25:51,720 --> 00:25:56,759
one key concept from earlier was that we

619
00:25:54,480 --> 00:25:58,140
created a single unit to process an

620
00:25:56,759 --> 00:26:00,359
input word

621
00:25:58,140 --> 00:26:02,340
and then we just copied that unit for

622
00:26:00,359 --> 00:26:04,859
each word in the input

623
00:26:02,339 --> 00:26:07,439
and if we had more words we just make

624
00:26:04,859 --> 00:26:09,839
more copies of the same unit

625
00:26:07,440 --> 00:26:12,538
by creating a single unit that can be

626
00:26:09,839 --> 00:26:14,099
copied for each input word the

627
00:26:12,538 --> 00:26:16,259
Transformer can do all of the

628
00:26:14,099 --> 00:26:18,359
computation for each word in the input

629
00:26:16,259 --> 00:26:20,700
at the same time

630
00:26:18,359 --> 00:26:22,979
for example we can calculate the word

631
00:26:20,700 --> 00:26:24,298
embeddings on different processors at

632
00:26:22,980 --> 00:26:26,759
the same time

633
00:26:24,298 --> 00:26:28,259
and then add the positional encoding at

634
00:26:26,759 --> 00:26:30,900
the same time

635
00:26:28,259 --> 00:26:33,119
and then calculate the queries keys and

636
00:26:30,900 --> 00:26:35,940
values at the same time

637
00:26:33,119 --> 00:26:38,038
and once that is done we can calculate

638
00:26:35,940 --> 00:26:39,120
the self-attention values at the same

639
00:26:38,038 --> 00:26:42,359
time

640
00:26:39,119 --> 00:26:44,819
and lastly we can calculate the residual

641
00:26:42,359 --> 00:26:47,158
connections at the same time

642
00:26:44,819 --> 00:26:49,019
doing all of the computations at the

643
00:26:47,159 --> 00:26:51,360
same time rather than doing them

644
00:26:49,019 --> 00:26:53,579
sequentially for each word

645
00:26:51,359 --> 00:26:56,099
means we can process a lot of words

646
00:26:53,579 --> 00:26:59,158
relatively quickly on a chip with a lot

647
00:26:56,099 --> 00:27:02,099
of computing cores like a GPU Graphics

648
00:26:59,159 --> 00:27:03,240
Processing Unit or multiple chips in the

649
00:27:02,099 --> 00:27:06,178
cloud

650
00:27:03,240 --> 00:27:08,460
well likewise when we decode and

651
00:27:06,179 --> 00:27:10,200
translate the input we want a single

652
00:27:08,460 --> 00:27:12,960
unit that we can copy for each

653
00:27:10,200 --> 00:27:15,120
translated word for the same reasons we

654
00:27:12,960 --> 00:27:18,120
want to do the math quickly

655
00:27:15,119 --> 00:27:21,599
so even though we're only processing the

656
00:27:18,119 --> 00:27:24,778
EOS token so far we add a self-attention

657
00:27:21,599 --> 00:27:27,538
layer so that ultimately we can keep

658
00:27:24,778 --> 00:27:30,058
track of related words in the output

659
00:27:27,538 --> 00:27:33,240
now that we have the query key and value

660
00:27:30,058 --> 00:27:36,418
numbers for the EOS token we calculate

661
00:27:33,240 --> 00:27:38,579
itself attention values just like before

662
00:27:36,419 --> 00:27:41,880
and the self-attention values for the

663
00:27:38,579 --> 00:27:46,079
EOS token are negative 2.8 and negative

664
00:27:41,880 --> 00:27:48,000
2.3 note the sets of Weights we used to

665
00:27:46,079 --> 00:27:50,879
calculate the decoder self-attention

666
00:27:48,000 --> 00:27:53,339
query key and value are different from

667
00:27:50,880 --> 00:27:56,580
the sets we used in the encoder

668
00:27:53,339 --> 00:27:59,519
now let's consolidate the math and add

669
00:27:56,579 --> 00:28:01,139
residual connections just like before

670
00:27:59,519 --> 00:28:03,599
bam

671
00:28:01,140 --> 00:28:05,640
now so far we've talked about how

672
00:28:03,599 --> 00:28:07,459
self-attention helps the Transformer

673
00:28:05,640 --> 00:28:09,900
keep track of how words are related

674
00:28:07,460 --> 00:28:12,179
within a sentence

675
00:28:09,900 --> 00:28:14,519
however since We're translating a

676
00:28:12,179 --> 00:28:16,860
sentence we also need to keep track of

677
00:28:14,519 --> 00:28:18,900
the relationships between the input

678
00:28:16,859 --> 00:28:22,439
sentence and the output

679
00:28:18,900 --> 00:28:25,259
for example if the input sentence was

680
00:28:22,440 --> 00:28:28,620
don't eat the delicious looking and

681
00:28:25,259 --> 00:28:31,140
smelling pizza then when translating

682
00:28:28,619 --> 00:28:33,658
it's super important to keep track of

683
00:28:31,140 --> 00:28:36,120
the very first word don't

684
00:28:33,659 --> 00:28:38,159
if the translation focuses on other

685
00:28:36,119 --> 00:28:40,439
parts of the sentence and omits the

686
00:28:38,159 --> 00:28:43,140
don't then we'll end up with

687
00:28:40,440 --> 00:28:44,159
eat the delicious looking and smelling

688
00:28:43,140 --> 00:28:46,799
Pizza

689
00:28:44,159 --> 00:28:48,600
and these two sentences have completely

690
00:28:46,798 --> 00:28:51,119
opposite meanings

691
00:28:48,599 --> 00:28:53,038
so it's super important for the decoder

692
00:28:51,119 --> 00:28:54,719
to keep track of the significant words

693
00:28:53,038 --> 00:28:57,720
in the input

694
00:28:54,720 --> 00:29:00,298
so the main idea of encoder decoder

695
00:28:57,720 --> 00:29:02,640
attention is to allow the decoder to

696
00:29:00,298 --> 00:29:04,259
keep track of the significant words in

697
00:29:02,640 --> 00:29:06,720
the input

698
00:29:04,259 --> 00:29:09,658
now that we know the main idea behind

699
00:29:06,720 --> 00:29:10,860
encoder decoder attention here are the

700
00:29:09,659 --> 00:29:13,020
details

701
00:29:10,859 --> 00:29:15,058
first to give us a little more room

702
00:29:13,019 --> 00:29:16,200
let's consolidate the math and the

703
00:29:15,058 --> 00:29:18,839
diagrams

704
00:29:16,200 --> 00:29:21,419
now just like we did for self-attention

705
00:29:18,839 --> 00:29:24,298
we create two new values to represent

706
00:29:21,419 --> 00:29:27,480
the query for the EOS token in the

707
00:29:24,298 --> 00:29:30,599
decoder then we create keys for each

708
00:29:27,480 --> 00:29:33,360
word in the encoder and we calculate the

709
00:29:30,599 --> 00:29:36,778
similarities between the EOS token in

710
00:29:33,359 --> 00:29:39,178
the decoder and each word in the encoder

711
00:29:36,778 --> 00:29:40,440
by calculating the dot products just

712
00:29:39,179 --> 00:29:42,720
like before

713
00:29:40,440 --> 00:29:45,659
then we run the similarities through a

714
00:29:42,720 --> 00:29:47,579
softmax function and this tells us to

715
00:29:45,659 --> 00:29:50,220
use one hundred percent of the first

716
00:29:47,579 --> 00:29:53,278
input word and zero percent of the

717
00:29:50,220 --> 00:29:56,038
second when the decoder determines what

718
00:29:53,278 --> 00:29:58,619
should be the first translated word

719
00:29:56,038 --> 00:30:01,019
now that we know what percentage of each

720
00:29:58,619 --> 00:30:03,658
input word to use when determining what

721
00:30:01,019 --> 00:30:07,319
should be the first translated word

722
00:30:03,659 --> 00:30:09,778
we calculate values for each input word

723
00:30:07,319 --> 00:30:11,759
and then scale those values by the soft

724
00:30:09,778 --> 00:30:14,099
Max percentages

725
00:30:11,759 --> 00:30:16,500
and then add the pairs of scaled values

726
00:30:14,099 --> 00:30:18,178
together to get the encoder decoder

727
00:30:16,500 --> 00:30:19,558
attention values

728
00:30:18,179 --> 00:30:22,259
bam

729
00:30:19,558 --> 00:30:24,240
now to make room for the next step let's

730
00:30:22,259 --> 00:30:26,339
consolidate the encoder decoder

731
00:30:24,240 --> 00:30:28,919
attention in our diagram

732
00:30:26,339 --> 00:30:31,139
note the sets of Weights that we use to

733
00:30:28,919 --> 00:30:33,778
calculate the queries keys and values

734
00:30:31,140 --> 00:30:35,520
for encoder decoder attention are

735
00:30:33,778 --> 00:30:37,380
different from the sets of Weights we

736
00:30:35,519 --> 00:30:40,319
use for self-attention

737
00:30:37,380 --> 00:30:42,600
however just like for self-attention the

738
00:30:40,319 --> 00:30:44,158
sets of Weights are copied and reused

739
00:30:42,599 --> 00:30:46,259
for each word

740
00:30:44,159 --> 00:30:48,480
this allows the Transformer to be

741
00:30:46,259 --> 00:30:50,220
flexible with the length of the inputs

742
00:30:48,480 --> 00:30:53,339
and outputs

743
00:30:50,220 --> 00:30:55,140
and also we can stack encode or decoder

744
00:30:53,339 --> 00:30:58,079
attention just like we can stack

745
00:30:55,140 --> 00:31:01,080
self-attention to keep track of words in

746
00:30:58,079 --> 00:31:03,480
complicated phrases bam

747
00:31:01,079 --> 00:31:04,678
now we add another set of residual

748
00:31:03,480 --> 00:31:07,319
connections

749
00:31:04,679 --> 00:31:09,480
that allow the encoder decoder attention

750
00:31:07,319 --> 00:31:11,519
to focus on the relationships between

751
00:31:09,480 --> 00:31:13,440
the output words and the input words

752
00:31:11,519 --> 00:31:15,720
without having to preserve the

753
00:31:13,440 --> 00:31:19,140
self-attention or word and position

754
00:31:15,720 --> 00:31:21,839
encoding that happened earlier then we

755
00:31:19,140 --> 00:31:24,720
consolidate the math and the diagram

756
00:31:21,839 --> 00:31:27,359
lastly we need a way to take these two

757
00:31:24,720 --> 00:31:30,419
values that represent the EOS token in

758
00:31:27,359 --> 00:31:35,339
the decoder and select one of the four

759
00:31:30,419 --> 00:31:37,620
output tokens ear vamos e or EOS

760
00:31:35,339 --> 00:31:39,839
so we run these two values through a

761
00:31:37,619 --> 00:31:41,819
fully connected layer that has one input

762
00:31:39,839 --> 00:31:44,398
for each value that represents the

763
00:31:41,819 --> 00:31:45,898
current token so in this case we have

764
00:31:44,398 --> 00:31:48,418
two inputs

765
00:31:45,898 --> 00:31:50,639
and one output for each token in the

766
00:31:48,419 --> 00:31:52,860
output vocabulary which in this case

767
00:31:50,640 --> 00:31:55,679
means four outputs

768
00:31:52,859 --> 00:31:57,418
note a fully connected layer is just a

769
00:31:55,679 --> 00:32:00,298
simple neural network with weights

770
00:31:57,419 --> 00:32:03,600
numbers we multiply the inputs by

771
00:32:00,298 --> 00:32:04,918
and biases numbers we add to the sums of

772
00:32:03,599 --> 00:32:07,139
the products

773
00:32:04,919 --> 00:32:08,880
and when we do the math we get four

774
00:32:07,140 --> 00:32:11,038
output values

775
00:32:08,880 --> 00:32:13,500
which we run through a final soft Max

776
00:32:11,038 --> 00:32:15,679
function to select the first output word

777
00:32:13,500 --> 00:32:17,519
vamos

778
00:32:15,679 --> 00:32:20,460
bam

779
00:32:17,519 --> 00:32:25,980
note vamos is the Spanish translation

780
00:32:20,460 --> 00:32:28,919
for Let's Go triple boom no not yet

781
00:32:25,980 --> 00:32:30,899
so far the translation is correct but

782
00:32:28,919 --> 00:32:34,080
the decoder doesn't stop until it

783
00:32:30,898 --> 00:32:36,298
outputs an EOS token so let's

784
00:32:34,079 --> 00:32:39,480
consolidate our diagrams

785
00:32:36,298 --> 00:32:41,879
and plug the translated word vamos into

786
00:32:39,480 --> 00:32:43,620
a copy of the decoder's embedding layer

787
00:32:41,880 --> 00:32:45,840
and do the math

788
00:32:43,619 --> 00:32:47,038
first we get the word embeddings for

789
00:32:45,839 --> 00:32:50,038
vamos

790
00:32:47,038 --> 00:32:52,440
then we add the positional encoding

791
00:32:50,038 --> 00:32:54,898
now we calculate self-attention values

792
00:32:52,440 --> 00:32:57,659
for vamos using the exact same weights

793
00:32:54,898 --> 00:33:00,418
that we used for the EOS token

794
00:32:57,659 --> 00:33:02,760
now add the residual connections

795
00:33:00,419 --> 00:33:05,038
and calculate the encoder decoder

796
00:33:02,759 --> 00:33:07,798
attention using the same sets of Weights

797
00:33:05,038 --> 00:33:10,679
that we used for the EOS token

798
00:33:07,798 --> 00:33:13,139
now we add more residual connections

799
00:33:10,679 --> 00:33:15,298
lastly we run the values that represent

800
00:33:13,140 --> 00:33:17,940
vamos through the same fully connected

801
00:33:15,298 --> 00:33:21,240
layer and softmax that we used for the

802
00:33:17,940 --> 00:33:25,259
EOS token and the second output from the

803
00:33:21,240 --> 00:33:27,960
decoder is eos so we are done decoding

804
00:33:25,259 --> 00:33:30,480
triple bam

805
00:33:27,960 --> 00:33:32,700
at long last we've shown how a

806
00:33:30,480 --> 00:33:34,798
Transformer can encode a simple input

807
00:33:32,700 --> 00:33:37,500
phrase let's go

808
00:33:34,798 --> 00:33:40,139
and decode the encoding into the

809
00:33:37,500 --> 00:33:41,778
translated phrase of vamos

810
00:33:40,140 --> 00:33:44,159
in summary

811
00:33:41,778 --> 00:33:46,259
Transformers use word embedding to

812
00:33:44,159 --> 00:33:48,720
convert words into numbers

813
00:33:46,259 --> 00:33:50,058
positional encoding to keep track of

814
00:33:48,720 --> 00:33:52,380
word order

815
00:33:50,058 --> 00:33:54,720
self-attention to keep track of word

816
00:33:52,380 --> 00:33:56,278
relationships within the input and

817
00:33:54,720 --> 00:33:59,038
output phrases

818
00:33:56,278 --> 00:34:01,259
encoder decoder attention to keep track

819
00:33:59,038 --> 00:34:03,480
of things between the input and output

820
00:34:01,259 --> 00:34:05,940
phrases to make sure that important

821
00:34:03,480 --> 00:34:07,200
words in the input are not lost in the

822
00:34:05,940 --> 00:34:09,599
translation

823
00:34:07,200 --> 00:34:12,599
and residual connections to allow each

824
00:34:09,599 --> 00:34:15,300
subunit like self-attention to focus on

825
00:34:12,599 --> 00:34:18,000
solving just one part of the problem

826
00:34:15,300 --> 00:34:20,460
now that we understand the main ideas of

827
00:34:18,000 --> 00:34:22,858
how Transformers work let's talk about a

828
00:34:20,460 --> 00:34:25,320
few extra things we can add to them

829
00:34:22,858 --> 00:34:26,398
in this example we kept things super

830
00:34:25,320 --> 00:34:29,039
simple

831
00:34:26,398 --> 00:34:31,980
however if we had larger vocabularies

832
00:34:29,039 --> 00:34:34,320
and the original Transformer had 37 000

833
00:34:31,980 --> 00:34:35,519
tokens and longer input and output

834
00:34:34,320 --> 00:34:37,980
phrases

835
00:34:35,519 --> 00:34:40,259
then in order to get their model to work

836
00:34:37,980 --> 00:34:41,760
they had to normalize the values after

837
00:34:40,260 --> 00:34:44,280
every step

838
00:34:41,760 --> 00:34:46,139
for example they normalize the values

839
00:34:44,280 --> 00:34:48,659
after positional encoding and after

840
00:34:46,139 --> 00:34:51,780
self-attention in both the encoder and

841
00:34:48,659 --> 00:34:54,480
the decoder also when we calculated

842
00:34:51,780 --> 00:34:57,060
attention values we used the dot product

843
00:34:54,480 --> 00:34:59,280
to calculate the similarities but you

844
00:34:57,059 --> 00:35:00,299
can use whatever similarity function you

845
00:34:59,280 --> 00:35:02,760
want

846
00:35:00,300 --> 00:35:05,099
in the original Transformer manuscript

847
00:35:02,760 --> 00:35:07,320
they calculated the similarities with a

848
00:35:05,099 --> 00:35:09,300
DOT product divided by the square root

849
00:35:07,320 --> 00:35:12,180
of the number of embedding values per

850
00:35:09,300 --> 00:35:14,400
token just like with scaling the values

851
00:35:12,179 --> 00:35:16,858
after each step they found that scaling

852
00:35:14,400 --> 00:35:19,740
the dot product helped encode and decode

853
00:35:16,858 --> 00:35:22,259
long and complicated phrases

854
00:35:19,739 --> 00:35:24,539
lastly to give a Transformer more

855
00:35:22,260 --> 00:35:26,940
weights and biases to fit to complicated

856
00:35:24,539 --> 00:35:28,800
data you can add additional neural

857
00:35:26,940 --> 00:35:31,858
networks with hidden layers to both the

858
00:35:28,800 --> 00:35:34,039
encoder and decoder bam

""".strip()

response = generate_response(prompt, max_new_tokens=1024)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.18 GiB (GPU 0; 15.90 GiB total capacity; 13.36 GiB already allocated; 1.33 GiB free; 13.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
print(response)

NameError: name 'response' is not defined